<H1><center>Neural Architecture Search using MCGRAN</center></H1>

# Multi-conditional GRAN

We model the conditional graph generation as an affine transformation based on given contraints.
We use $ \textrm{MLP}_{scale} $ and $ \textrm{MLP}_{shift} $ to geometrically transform the feature space of bernoulli mixture components.
Our intention with this transformation is to separate the various constraints(classes or categories) of graphs into distinct real space.

\begin{align}
	\alpha_{1},..., \alpha_{K} = \textrm{Softmax}(\sum_{i \in \boldsymbol{b}_{t},1 \leq j \leq i} \textrm{MLP}_{\alpha} (h_{i}^{R} - h_{j}^{R}) \otimes \textrm{MLP}_{scale}(c_{i}) + \textrm{MLP}_{shift}(c_{i}))
\end{align}

\begin{align}
	\theta_{1,i,j},..., \theta_{K,i,j} = \textrm{Sigmoid}(\textrm{MLP}_{\theta} (h_{i}^{R} - h_{j}^{R})  \otimes \textrm{MLP}_{scale}(c_{i}) + \textrm{MLP}_{shift}(c_{i}))
\end{align}

In equations, $ M $ denote the number of constraints enforced on each node $ i $, $ c_{i} \in \mathbb{R}^{M} $ represents the constraints vector associated with each node $ i $, $ \textrm{MLP}_{scale} \in  \mathbb{R}^{K \times H}$ is a RELU-based hidden layer capturing features for scaling factor, and $ \textrm{MLP}_{shift} \in  \mathbb{R}^{K \times H} $ is a RELU-based hidden layer capturing features for shift factor. $ K $ denote the number of mixture components and $ H $ denote the hidden dimension size.

# Graph-based Auto-regressive Affine Transformations

Using the structure of the graph, that is, the adjacency matrix $ A $ for each graph $G = (V,E)$. We create node level features using 1D convolutions. We use a block of three 1D convolutional hidden layers stacked together to capture features for each vertices $ v $, where $ \textrm{CNN}(A): \mathbb{R}^{|V| \times |V|} \rightarrow \mathbb{R}^{|V| \times H}$, where adjacency matrix denoted as $ A \in \mathbb{R}^{|V| \times |V|} $. Each channel of convolution captures the information specific to a node.

Then, we apply auto-regressive affine transformation to create node labels.  In graph-based auto-regressive affine transformations, for each node label prediction, we use three information.  First information comes from the node features which we captured using the CNN. Let us denote node features for a node $v\in V$ as $q_v\in\mathbb{R}^{H}$ with $H$ being the feature dimension.

Second information comes from the nodes features of the other connected nodes. This acts as neighbor constraints to predict the current node label. For a node $v\in V$ of a graph $G = (V,E)$ its set of neighboring features is given as
\[
Neigh(v) = \{ q_s ~|~ (s,v) \in E \}
\]
We could also define $NAgg(v): \mathbb{R}^{|Neigh(v)| \times H} \rightarrow \mathbb{R}^{H}$

\begin{align}
	NAgg(v) = \sum\limits_{q\in\{ q_s ~|~ (s,v) \in E \}}
\end{align}

For each node $ v $, we apply affine transformation based on node neighbors with one scaling factor $ \textrm{MLP}_{scale}(NAgg(v)) $ and one shift factor $ \textrm{MLP}_{shift}(NAgg(v)) $. 

Third information comes from the graph level constraint. We apply geometric transformation based on graph constraints with one scaling factor  $ \textrm{MLP}_{scale}(c_{v}) $ and one shift factor $ \textrm{MLP}_{shift}(c_{v}) $.

The $ c_{v} \in \mathbb{R}^{M} $ represents the constraints vector associated with each node $ v $, the $ M $ denote the number of constraints enforced on each node $ v $, the $ \textrm{MLP}_{scale} \in  \mathbb{R}^{H}$ is a RELU-based hidden layer capturing features for scaling factor, and the $ \textrm{MLP}_{shift} \in  \mathbb{R}^{ H} $ is a RELU-based hidden layer capturing features for shift factor. The $ H $ denote the hidden dimension size.

%From a social network perspective, each person (node) uses the information he or she knows, then collects information from his or her friends, and finally collects information other sources (books or internet) to make a decision.
%The same process is applied here.

We can combine all three information to predict for each node $ v $ the label as shown in equation~\ref{eq:mcgran_node_label_equation}.

\begin{align}
	v = \textrm{Softmax}(\textrm{CNN}(v) \otimes \textrm{MLP}_{scale}(NAgg(v)) \otimes \textrm{MLP}_{scale}(c_{v})   + \textrm{MLP}_{shift}(NAgg(v)) + \textrm{MLP}_{shift}(c_{v}))
\end{align}

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |████████████████▎               | 834.1 MB 1.2 MB/s eta 0:11:18tcmalloc: large alloc 1147494400 bytes == 0x39b32000 @  0x7fb54d1fe615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |████████████████████▋           | 1055.7 MB 1.2 MB/s eta 0:07:57tcmalloc: large alloc 1434370048 bytes == 0x7e188000 @  0x7fb54d1fe615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |██████████████████████████▏     | 

In [2]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.2 MB/s 


In [3]:
!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498M  100  498M    0     0   152M      0  0:00:03  0:00:03 --:--:--  152M


In [4]:
#!g++ -O2 -std=c++11 -o drive/MyDrive/Research-NAS/MCGRAN/utils/orca/orca.cpp

In [5]:
#!chmod +x drive/MyDrive/Research-NAS/MCGRAN/utils/orca/orca

## Configuration Settings

All the configurable settings for working the GRAN model are specified in the yaml files located in the config folder.  There are five sections of parameters in the configuration files:

1.   General experimental parameters
2.   Dataset parameters
3.   Model parameters
4.   Training parameters
5.   Test parameters

### General experimental parameters

Name of the experiment
> **exp_name**: *MCGRAN*

The experiment directory folder name if not already present.  This folder will contain the training and evaluation metrics.

> **exp_dir**: *exp/MCGRAN/* 

Name of the runner class name from gran_runner_*.py file
> **runner**: *GranRunner_Evaluation*

Distributed training of the model in multiple machines. Always set to false.  We did not test with true.
> **use_horovod**: *false*

GRU related settings. Always set to true. we did not test with false.
> **use_gpu**: *true*      

Cuda device id
> **device**: *cuda:0*  

Number of GPUs
> **gpus**: [0]

Random seed for reproducing the experiments
> seed: 78123456

In [ ]:
!python drive/MyDrive/Research-NAS/MCGRAN/run_exp.py -c drive/MyDrive/Research-NAS/MCGRAN/config/mcgran.yaml -t

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 44 seconds
INFO  | 2022-12-08 12:21:14,435 | run_exp.py                | line 31   : Writing log file to /content/drive/MyDrive/Research-NAS/MCGRAN/exp/MCGRAN/503/log_exp_503.txt
INFO  | 2022-12-08 12:21:14,437 | run_exp.py                | line 32   : Exp instance id = 503
INFO  | 2022-12-08 12:21:14,438 | run_exp.py                | line 33   : Exp comment = None
INFO  | 2022-12-08 12:21:14,438 | run_exp.py                | line 34   : Config =
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
{'dataset': {'data_path': './',
             'dev_ratio': 0.2,
             'has_node_feat': False,
             'is_overwrite_precompute': False,
             'is_sample_subgraph': True,
             'is_save_split': Tr

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir 'drive/MyDrive/GRAN/exp/GRAN_EVALUATION'

# Evaluation

In [ ]:
import os
import pickle
import numpy as np

base_path = "drive/MyDrive/GRAN/exp/GRAN_EVALUATION"
experiment_folder_list = os.listdir(base_path)

We created 8 model with different random seeds and collected the data for the below evaluation metrics.

*   Self-loops
*   Isolated Nodes
*   Validity of generated graphs that satisfy constraints of Neural Network
*   Maximum Mean Discrepancy(MMD) of number of nodes, node degree, node clustering, and graph spectrum
*   Test Accuracy of generated neural networks
*   Uniqueness of generated neural networks
*   Novelty of generated neural networks
*   Visual analysis of general neural network

## Self-loops

A self-loop check determines if the node connects to itself. For neural architecture, we don't expect the layers to connect to themselves. We will count the graphs which have self-loops.

In [ ]:
structure_evaluation_metrics_list = list()
for folder_name in experiment_folder_list:
    folder_name = os.path.join(base_path, folder_name)
    file_name = os.path.join(folder_name, 'structure_evaluation_metrics.p')
    if os.path.exists(file_name):
        structure_evaluation_metrics = pickle.load(open(file_name, 'rb'))
        structure_evaluation_metrics_list.append(structure_evaluation_metrics)

In [ ]:
self_loops_list = list()
for metrics in structure_evaluation_metrics_list:
    self_loops_list.append(metrics['self_loops'])
self_loops_list = np.array(self_loops_list)
print("Maximum number of graphs with self-loops ", np.max(self_loops_list))
print("Minimum number of graphs with self-loops ", np.min(self_loops_list))
print("Average number of graphs with self-loops ", np.mean(self_loops_list))
print("Standard deviation of graphs with self-loops ", np.std(self_loops_list))

###Isolated Nodes

Isolated nodes check will identify nodes not connected to any other in the graph. In a neural network, we expect the layers to form connections with another layer. We will count the graphs with isolated nodes.

In [ ]:
isolated_nodes_list = list()
for metrics in structure_evaluation_metrics_list:
    isolated_nodes_list.append(metrics['isolated_nodes'])
isolated_nodes_list = np.array(isolated_nodes_list)
print("Maximum number of graphs with isolated nodes ", np.max(isolated_nodes_list))
print("Minimum number of graphs with isolated nodes ", np.min(isolated_nodes_list))
print("Average number of graphs with isolated nodes ", np.mean(isolated_nodes_list))
print("Standard deviation of graphs with isolated nodes ", np.std(isolated_nodes_list))

###Validity

Validity measures the number of validly generated graphs. Specifically, it has to satisfy the constraints of Directed Acyclic Graph and Neural Networks.

*   Only the input node is at the start.
*   Only the output node is at the end.
*   Except for the output node, there should not be any node without a successor.
*   Except for the input node, there should not be any node without a predecessor.









In [ ]:
invalid_nn_list = list()
for metrics in structure_evaluation_metrics_list:
    invalid_nn_list.append(metrics['invalid_nn'])
invalid_nn_list = np.array(invalid_nn_list)
print("Maximum number of invalid graphs generated ", np.max(invalid_nn_list))
print("Minimum number of invalid graphs generated ", np.min(invalid_nn_list))
print("Average number of invalid graphs generated ", np.mean(invalid_nn_list))
print("Standard deviation of invalid graphs generated ", np.std(invalid_nn_list))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
plt.boxplot([invalid_nn_list],
           labels=['Invalid graphs'])

###Maximum Mean Discrepancy

Maximum Mean Discrepancy(MMD) compares two probability distributions and reports how far they are apart. We evaluate the following metrics only on valid graphs in order to eliminate the discrepancy caused by invalid graphs.

\begin{equation}
    \operatorname{MMD}^{2}(p \| q) =\mathbb{E}_{x, y \sim p}[k(x, y)] + \mathbb{E}_{x, y \sim q}[k(x, y)] - 2 \mathbb{E}_{x \sim p, y \sim q}[k(x, y)]
\end{equation}


In [ ]:
Validation_MMD_Scores_Validation_vs_Generated_Graphs_list = list()
for folder_name in experiment_folder_list:
    folder_name = os.path.join(base_path, folder_name)
    file_name = os.path.join(folder_name, 'Validation_MMD_Scores_Validation_vs_Generated_Graphs.p')
    if os.path.exists(file_name):
        Validation_MMD_Scores_Validation_vs_Generated_Graphs = pickle.load(open(file_name, 'rb'))
        Validation_MMD_Scores_Validation_vs_Generated_Graphs_list.append(Validation_MMD_Scores_Validation_vs_Generated_Graphs)

In [ ]:
MMD_num_of_nodes_list = list()
for metrics in Validation_MMD_Scores_Validation_vs_Generated_Graphs_list:
    MMD_num_of_nodes_list.append(metrics[0])
MMD_num_of_nodes_list = np.array(MMD_num_of_nodes_list)
print("Maximum MMD distance between number of nodes in the validation and generated set ", np.max(MMD_num_of_nodes_list))
print("Minimum MMD distance between number of nodes in the validation and generated set ", np.min(MMD_num_of_nodes_list))
print("Average MMD distance between number of nodes in the validation and generated set ", np.mean(MMD_num_of_nodes_list))
print("Standard deviation of MMD distance between number of nodes in the validation and generated set ", np.std(MMD_num_of_nodes_list))

In [ ]:
MMD_node_degree_list = list()
for metrics in Validation_MMD_Scores_Validation_vs_Generated_Graphs_list:
    MMD_node_degree_list.append(metrics[1])
MMD_node_degree_list = np.array(MMD_node_degree_list)
print("Maximum MMD distance between node degree in the validation and generated set ", np.max(MMD_node_degree_list))
print("Minimum MMD distance between node degree in the validation and generated set ", np.min(MMD_node_degree_list))
print("Average MMD distance between node degree in the validation and generated set ", np.mean(MMD_node_degree_list))
print("Standard deviation of MMD distance between node degree in the validation and generated set ", np.std(MMD_node_degree_list))

In [ ]:
MMD_node_clustering_list = list()
for metrics in Validation_MMD_Scores_Validation_vs_Generated_Graphs_list:
    MMD_node_clustering_list.append(metrics[2])
MMD_node_clustering_list = np.array(MMD_node_clustering_list)
print("Maximum MMD distance between node clustering in the validation and generated set ", np.max(MMD_node_clustering_list))
print("Minimum MMD distance between node clustering in the validation and generated set ", np.min(MMD_node_clustering_list))
print("Average MMD distance between node clustering in the validation and generated set ", np.mean(MMD_node_clustering_list))
print("Standard deviation of MMD distance between node clustering in the validation and generated set ", np.std(MMD_node_clustering_list))

In [ ]:
MMD_node_orbits_list = list()
for metrics in Validation_MMD_Scores_Validation_vs_Generated_Graphs_list:
    MMD_node_orbits_list.append(metrics[3])
MMD_node_orbits_list = np.array(MMD_node_orbits_list)
print("Maximum MMD distance between node orbit count in the validation and generated set ", np.max(MMD_node_orbits_list))
print("Minimum MMD distance between node orbit count in the validation and generated set ", np.min(MMD_node_orbits_list))
print("Average MMD distance between node orbit count in the validation and generated set ", np.mean(MMD_node_orbits_list))
print("Standard deviation of MMD distance between node orbit count in the validation and generated set ", np.std(MMD_node_orbits_list))

In [ ]:
MMD_graph_spectrum_list = list()
for metrics in Validation_MMD_Scores_Validation_vs_Generated_Graphs_list:
    MMD_graph_spectrum_list.append(metrics[4])
MMD_graph_spectrum_list = np.array(MMD_graph_spectrum_list)
print("Maximum MMD distance between graph laplacian in the validation and generated set ", np.max(MMD_graph_spectrum_list))
print("Minimum MMD distance between graph laplacian in the validation and generated set ", np.min(MMD_graph_spectrum_list))
print("Average MMD distance between graph laplacian in the validation and generated set ", np.mean(MMD_graph_spectrum_list))
print("Standard deviation of MMD distance between graph laplacian in the validation and generated set ", np.std(MMD_graph_spectrum_list))

###Accuracy

Let us review the the test accuracy of the valid graphs. In the following list, each value is the mean of the test accuracy of the generated graphs for the GRAN model generated with a random seed.

In [ ]:
test_accuracy_list = list()
for folder_name in experiment_folder_list:
    folder_name = os.path.join(base_path, folder_name)
    file_name = os.path.join(folder_name, 'generated_graph_test_accuracy_list.p')
    if os.path.exists(file_name):
        generated_graph_test_accuracy_list = pickle.load(open(file_name, 'rb'))
        test_accuracy_list.append(sum(generated_graph_test_accuracy_list)/len(generated_graph_test_accuracy_list))
test_accuracy_list = np.array(test_accuracy_list)
print("Maximum test accuracy of valid graphs generated ", np.max(test_accuracy_list))
print("Minimum test accuracy of valid graphs generated ", np.min(test_accuracy_list))
print("Average test accuracy of valid graphs generated ", np.mean(test_accuracy_list))
print("Standard deviation of the test accuracy of valid graphs generated ", np.std(test_accuracy_list))

###Uniqueness

Uniqueness measures the unique share of the generated graphs. Simply put, we calculate the graph similarity. If the generated graphs are diverse among the generated graphs, then the generative model produces unique graphs.

Graph edit distance measure tells us the minimum of edits in nodes and edges required in a graph to produce a corresponding isomorphic graph.

In [ ]:
uniqueness_list = list()
for metrics in structure_evaluation_metrics_list:
    if 'uniqueness' in metrics.keys():
        uniqueness_list.append(metrics['uniqueness'])

graph_edit_distance_distribution = dict()
for metric in uniqueness_list:
    for key in metric.keys():
        if key in graph_edit_distance_distribution.keys():
            graph_edit_distance_distribution[key] += metric[key]
        else:
            graph_edit_distance_distribution[key] = metric[key]

for key in graph_edit_distance_distribution.keys():
    graph_edit_distance_distribution[key] = graph_edit_distance_distribution[key]/8 # 8 represent the number of evaluations

for key in sorted(graph_edit_distance_distribution):
    print(f"{key}: {graph_edit_distance_distribution[key]}")

###Novelty

We measure novelty as the valid number of generated graphs not available in the training dataset.

As defined earlier, graph edit distance measure the number of changes required to make one graph isomorphic to another graph. The graph edit distance metric will measure the novelty as the number of changes in the generated graphs to create the same graph in the training set. Therefore, the more the graph edit distance, the more the novelty of the graph generation model.

In [ ]:
novelty_list = list()
for metrics in structure_evaluation_metrics_list:
    if 'novelty' in metrics.keys():
        novelty_list.append(metrics['novelty'])
graph_edit_distance_distribution = dict()
for metric in novelty_list:
    for key in metric.keys():
        if key in graph_edit_distance_distribution.keys():
            graph_edit_distance_distribution[key] += metric[key]
        else:
            graph_edit_distance_distribution[key] = metric[key]
for key in graph_edit_distance_distribution.keys():
    graph_edit_distance_distribution[key] = graph_edit_distance_distribution[key]/8 # 8 represent the number of evaluations
for key in sorted(graph_edit_distance_distribution):
    print(f"{key}: {graph_edit_distance_distribution[key]}")

###Visual Analysis using PyDot

As the visual analysis of the all the graphs in this notebook is not possible. We provide a simple interface to visualize the generated graphs.

In [ ]:
from IPython.display import SVG
selected_experiment_folder_name = 'GRANMixtureBernoulli_Evaluation_nas_2022-Jan-03-11-41-31_30198'
selected_experiment_folder_name = os.path.join(base_path, selected_experiment_folder_name)

In [ ]:
selected_graph_name = "architecture_3.svg" # graphs are numbered from 0 to 99
file_name = os.path.join(selected_experiment_folder_name, selected_graph_name)
SVG(file_name)

In [ ]:
selected_graph_name = "architecture_4.svg" # graphs are numbered from 0 to 99
file_name = os.path.join(selected_experiment_folder_name, selected_graph_name)
SVG(file_name)